<a href="https://colab.research.google.com/github/SamiraSamrose/wfip-platform/blob/main/WFIP_web_feature_intelligence_platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Installing dependencies...")

!pip install -q fastapi uvicorn aiohttp pydantic python-multipart nest-asyncio
!pip install -q playwright
!playwright install chromium
!pip install -q pyngrok

print("Dependencies installed!")

Installing dependencies...
Dependencies installed!


In [ ]:
print("Setting up ngrok for public access...")

from pyngrok import ngrok
import nest_asyncio

nest_asyncio.apply()

NGROK_TOKEN = "xyz"

if NGROK_TOKEN != "YOUR_NGROK_TOKEN":
    ngrok.set_auth_token(NGROK_TOKEN)
    print("ngrok configured!")
else:
    print("  Please set your ngrok token in the code above")
    print("  Get free token from: https://dashboard.ngrok.com/get-started/your-authtoken")


Setting up ngrok for public access...
ngrok configured!


In [ ]:
print(" Setting up WFIP server...")

import json
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
from datetime import datetime
from collections import defaultdict
import asyncio

from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# ============================================================================
# DATA MODELS
# ============================================================================

@dataclass
class WebFeature:
    name: str
    baseline_status: str
    global_support: float
    safe_year: Optional[int]
    alternatives: List[str]
    browsers: Dict[str, str]
    mdn_url: Optional[str] = None

@dataclass
class FeatureUsage:
    feature_name: str
    file_path: str
    line_number: int
    code_snippet: str

@dataclass
class RiskScore:
    feature_name: str
    risk_level: float
    global_support: float
    affected_markets: List[Tuple[str, float]]
    safe_year: Optional[int]
    alternatives: List[str]
    recommendation: str
    browsers: Dict[str, str]

# Pydantic models for API
class ScanRequest(BaseModel):
    code: str
    ui_name: Optional[str] = "Demo UI"

class FeatureRiskRequest(BaseModel):
    feature_name: str

# ============================================================================
# MOCK DATA STORE (Simplified for Colab)
# ============================================================================

class BaselineDataStore:
    """Mock baseline data for demo"""

    def __init__(self):
        self.features = self._load_mock_data()

    def _load_mock_data(self) -> Dict[str, WebFeature]:
        return {
            "backdrop-filter": WebFeature(
                name="backdrop-filter",
                baseline_status="widely_available",
                global_support=94.5,
                safe_year=2022,
                alternatives=["filter + position:fixed fallback"],
                browsers={"chrome": "76", "safari": "9", "firefox": "103"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/backdrop-filter"
            ),
            "scroll-snap": WebFeature(
                name="scroll-snap",
                baseline_status="widely_available",
                global_support=96.2,
                safe_year=2021,
                alternatives=["scroll libraries"],
                browsers={"chrome": "69", "safari": "11", "firefox": "68"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/scroll-snap-type"
            ),
            ":has()": WebFeature(
                name=":has()",
                baseline_status="newly_available",
                global_support=87.3,
                safe_year=2024,
                alternatives=[":not() combinations", "JS-based selectors"],
                browsers={"chrome": "105", "safari": "15.4", "firefox": "121"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/:has"
            ),
            "container-queries": WebFeature(
                name="container-queries",
                baseline_status="newly_available",
                global_support=85.1,
                safe_year=2024,
                alternatives=["media queries", "ResizeObserver"],
                browsers={"chrome": "105", "safari": "16", "firefox": "110"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Container_Queries"
            ),
            "subgrid": WebFeature(
                name="subgrid",
                baseline_status="limited",
                global_support=72.4,
                safe_year=2026,
                alternatives=["nested grids", "flexbox"],
                browsers={"chrome": "117", "safari": "16", "firefox": "71"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Grid_Layout/Subgrid"
            ),
            "view-transitions": WebFeature(
                name="view-transitions",
                baseline_status="limited",
                global_support=68.2,
                safe_year=2026,
                alternatives=["CSS transitions", "FLIP animations"],
                browsers={"chrome": "111", "safari": "None", "firefox": "None"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/API/View_Transitions_API"
            ),
            "IntersectionObserver": WebFeature(
                name="IntersectionObserver",
                baseline_status="widely_available",
                global_support=96.8,
                safe_year=2019,
                alternatives=["scroll event listeners"],
                browsers={"chrome": "51", "safari": "12.1", "firefox": "55"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/API/Intersection_Observer_API"
            ),
            "@layer": WebFeature(
                name="@layer",
                baseline_status="newly_available",
                global_support=89.5,
                safe_year=2024,
                alternatives=["CSS specificity management"],
                browsers={"chrome": "99", "safari": "15.4", "firefox": "97"},
                mdn_url="https://developer.mozilla.org/en-US/docs/Web/CSS/@layer"
            ),
        }

    def get_feature(self, name: str) -> Optional[WebFeature]:
        return self.features.get(name)

    def get_all_features(self) -> List[WebFeature]:
        return list(self.features.values())

# ============================================================================
# FEATURE DETECTOR
# ============================================================================

class FeatureDetector:
    """Detects web features in code"""

    def __init__(self, baseline_store: BaselineDataStore):
        self.baseline_store = baseline_store
        self._build_patterns()

    def _build_patterns(self):
        self.patterns = {
            "backdrop-filter": r'backdrop-filter\s*:',
            "scroll-snap": r'scroll-snap-(?:type|align|stop)\s*:',
            ":has()": r':has\s*\(',
            "container-queries": r'@container\s+',
            "subgrid": r'(?:grid-template-(?:columns|rows)|grid)\s*:\s*[^;]*subgrid',
            "view-transitions": r'view-transition-name\s*:|document\.startViewTransition',
            "@layer": r'@layer\s+',
            "IntersectionObserver": r'new\s+IntersectionObserver',
        }

    def scan_code(self, code: str, file_path: str = "demo.css") -> List[FeatureUsage]:
        usages = []
        lines = code.split('\n')

        for feature_name, pattern in self.patterns.items():
            for line_num, line in enumerate(lines, 1):
                if re.search(pattern, line, re.IGNORECASE):
                    usages.append(FeatureUsage(
                        feature_name=feature_name,
                        file_path=file_path,
                        line_number=line_num,
                        code_snippet=line.strip()[:100]
                    ))

        return usages

# ============================================================================
# RISK SCORER
# ============================================================================

class RiskScorer:
    """Calculates risk scores"""

    def __init__(self, baseline_store: BaselineDataStore):
        self.baseline_store = baseline_store

    def score_feature(self, feature_name: str) -> Optional[RiskScore]:
        feature = self.baseline_store.get_feature(feature_name)
        if not feature:
            return None

        risk = self._calculate_risk(feature)
        affected_markets = self._get_affected_markets(feature.global_support)
        recommendation = self._generate_recommendation(feature, risk)

        return RiskScore(
            feature_name=feature.name,
            risk_level=risk,
            global_support=feature.global_support,
            affected_markets=affected_markets,
            safe_year=feature.safe_year,
            alternatives=feature.alternatives,
            recommendation=recommendation,
            browsers=feature.browsers
        )

    def _calculate_risk(self, feature: WebFeature) -> float:
        base_risk = (100 - feature.global_support) / 10

        multipliers = {
            "widely_available": 0.5,
            "newly_available": 1.0,
            "limited": 1.5
        }

        risk = base_risk * multipliers.get(feature.baseline_status, 1.0)
        return min(10.0, max(0.0, risk))

    def _get_affected_markets(self, support: float) -> List[Tuple[str, float]]:
        unsupported = 100 - support
        markets = [
            ("India", unsupported * 0.9),
            ("Brazil", unsupported * 0.8),
            ("China", unsupported * 0.85),
            ("Indonesia", unsupported * 0.75),
            ("Nigeria", unsupported * 0.7)
        ]
        return sorted(markets, key=lambda x: x[1], reverse=True)[:5]

    def _generate_recommendation(self, feature: WebFeature, risk: float) -> str:
        if risk < 3:
            return f"✅ Safe to use. {feature.name} has excellent support ({feature.global_support}%)"
        elif risk < 6:
            return f"⚠️ Use with caution. Consider progressive enhancement or polyfills"
        else:
            alts = ", ".join(feature.alternatives) if feature.alternatives else "none identified"
            return f"🔥 High risk. Strongly consider alternatives: {alts}"

# ============================================================================
# FASTAPI APP
# ============================================================================

app = FastAPI()

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize services
baseline_store = BaselineDataStore()
detector = FeatureDetector(baseline_store)
risk_scorer = RiskScorer(baseline_store)

# ============================================================================
# API ENDPOINTS
# ============================================================================

@app.get("/")
async def root():
    return {
        "message": " WFIP API - Web Feature Intelligence Platform",
        "status": "running",
        "version": "1.0.0",
        "endpoints": {
            "features": "/features",
            "scan": "/scan",
            "risk": "/risk/{feature_name}",
            "docs": "/docs"
        }
    }

@app.get("/features")
async def list_features():
    """List all tracked features"""
    features = baseline_store.get_all_features()
    return {
        "total": len(features),
        "features": [asdict(f) for f in features]
    }

@app.get("/features/{feature_name}")
async def get_feature_details(feature_name: str):
    """Get detailed information about a specific feature"""
    feature = baseline_store.get_feature(feature_name)
    if not feature:
        raise HTTPException(status_code=404, detail=f"Feature '{feature_name}' not found")
    return asdict(feature)

@app.post("/scan")
async def scan_code(request: ScanRequest):
    """Scan code for feature usage"""
    try:
        usages = detector.scan_code(request.code)

        # Calculate compatibility score
        unique_features = list(set(u.feature_name for u in usages))
        min_support = 100.0

        for feature_name in unique_features:
            feature = baseline_store.get_feature(feature_name)
            if feature:
                min_support = min(min_support, feature.global_support)

        # Categorize by risk
        risk_categories = {"high": [], "medium": [], "low": []}
        for feature_name in unique_features:
            risk_score = risk_scorer.score_feature(feature_name)
            if risk_score:
                if risk_score.risk_level >= 6:
                    risk_categories["high"].append(feature_name)
                elif risk_score.risk_level >= 3:
                    risk_categories["medium"].append(feature_name)
                else:
                    risk_categories["low"].append(feature_name)

        return {
            "status": "success",
            "ui_name": request.ui_name,
            "usages_found": len(usages),
            "unique_features": len(unique_features),
            "compatibility_score": {
                "global_support": min_support,
                "affected_users_pct": 100 - min_support,
                "features_by_risk": risk_categories
            },
            "usages": [asdict(u) for u in usages]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/risk/{feature_name}")
async def get_feature_risk(feature_name: str):
    """Get risk assessment for a specific feature"""
    risk = risk_scorer.score_feature(feature_name)
    if not risk:
        raise HTTPException(status_code=404, detail=f"Feature '{feature_name}' not found")
    return asdict(risk)

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "features_loaded": len(baseline_store.features),
        "timestamp": datetime.now().isoformat()
    }

print(" WFIP Server configured!")


 Setting up WFIP server...
 WFIP Server configured!


In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [ ]:
print(" Starting WFIP server...")
print("=" * 60)

import threading
import uvicorn
from pyngrok import ngrok
import time
import nest_asyncio
from fastapi import FastAPI

nest_asyncio.apply()

public_url = ngrok.connect(8000)
public_url = public_url.public_url
BASE = public_url

print(f" Public URL: {public_url}")
print(f" API Docs: {public_url}/docs")
print(f" Interactive API: {public_url}/docs")
print("=" * 60)

# Run server in background thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
time.sleep(3)

print(" Server is running!")
print("\n Quick Start:")
print("1. Click the 'API Docs' link above to explore endpoints")
print("2. Try the /features endpoint to see all tracked features")
print("3. Use /scan endpoint to analyze code")
print("4. Use /risk/{feature_name} to get risk scores")
print("\n" + "=" * 60)


 Starting WFIP server...


INFO:     Started server process [17005]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


 Public URL: https://raymond-overheady-allopathically.ngrok-free.dev
 API Docs: https://raymond-overheady-allopathically.ngrok-free.dev/docs
 Interactive API: https://raymond-overheady-allopathically.ngrok-free.dev/docs
 Server is running!

 Quick Start:
1. Click the 'API Docs' link above to explore endpoints
2. Try the /features endpoint to see all tracked features
3. Use /scan endpoint to analyze code
4. Use /risk/{feature_name} to get risk scores



In [ ]:
print("Registered routes in FastAPI:")
for route in app.routes:
    print(f" - {route.path} ({route.methods})")


Registered routes in FastAPI:
 - /openapi.json ({'HEAD', 'GET'})
 - /docs ({'HEAD', 'GET'})
 - /docs/oauth2-redirect ({'HEAD', 'GET'})
 - /redoc ({'HEAD', 'GET'})
 - / ({'GET'})
 - /features ({'GET'})
 - /features/{feature_name} ({'GET'})
 - /scan ({'POST'})
 - /risk/{feature_name} ({'GET'})
 - /health ({'GET'})


In [ ]:
print(" Demo 1: Listing All Features\n")

import requests

try:
    response = requests.get(f"{public_url}/features")
    data = response.json()

    print(f" Found {data['total']} features:\n")

    for feature in data['features'][:5]:
        print(f" {feature['name']}")
        print(f"   Support: {feature['global_support']}%")
        print(f"   Risk Level: {(100 - feature['global_support']) / 10:.1f}/10")
        print(f"   Status: {feature['baseline_status']}")
        print(f"   Browsers: {feature['browsers']}")
        print()

    print(f"... and {data['total'] - 5} more features")

except Exception as e:
    print(f" Error: {e}")
    print("Make sure the server is running (run previous cell)")

 Demo 1: Listing All Features

INFO:     34.106.104.187:0 - "GET /features HTTP/1.1" 200 OK
 Found 8 features:

 backdrop-filter
   Support: 94.5%
   Risk Level: 0.6/10
   Status: widely_available
   Browsers: {'chrome': '76', 'safari': '9', 'firefox': '103'}

 scroll-snap
   Support: 96.2%
   Risk Level: 0.4/10
   Status: widely_available
   Browsers: {'chrome': '69', 'safari': '11', 'firefox': '68'}

 :has()
   Support: 87.3%
   Risk Level: 1.3/10
   Status: newly_available
   Browsers: {'chrome': '105', 'safari': '15.4', 'firefox': '121'}

 container-queries
   Support: 85.1%
   Risk Level: 1.5/10
   Status: newly_available
   Browsers: {'chrome': '105', 'safari': '16', 'firefox': '110'}

 subgrid
   Support: 72.4%
   Risk Level: 2.8/10
   Status: limited
   Browsers: {'chrome': '117', 'safari': '16', 'firefox': '71'}

... and 3 more features


In [ ]:
print(" Demo 2: Scanning Code for Features\n")

# Sample CSS code with various features
sample_code = """
.header {
    backdrop-filter: blur(10px);
    background: rgba(255, 255, 255, 0.8);
}

.container:has(> .active) {
    background: blue;
}

@container (min-width: 400px) {
    .card {
        display: grid;
        grid-template-columns: subgrid;
    }
}

.scroll-container {
    scroll-snap-type: y mandatory;
}
"""

print("Code to scan:")
print("-" * 60)
print(sample_code)
print("-" * 60)

try:
    response = requests.post(f"{public_url}/scan", json={"code": sample_code, "ui_name": "Demo UI"})
    data = response.json()

    print(f"\n✅ Scan Results:")
    print(f"   Features Found: {data['usages_found']}")
    print(f"   Unique Features: {data['unique_features']}")
    print(f"   Global Support: {data['compatibility_score']['global_support']:.1f}%")
    print(f"   Affected Users: {data['compatibility_score']['affected_users_pct']:.1f}%\n")

    print("📊 Features by Risk:")
    risk_data = data['compatibility_score']['features_by_risk']
    print(f"   🔴 High Risk: {', '.join(sorted(risk_data['high'])) if risk_data['high'] else 'None'}")
    print(f"   🟡 Medium Risk: {', '.join(sorted(risk_data['medium'])) if risk_data['medium'] else 'None'}")
    print(f"   🟢 Low Risk: {', '.join(sorted(risk_data['low'])) if risk_data['low'] else 'None'}")


    print("\n📋 Detected Features:")
    for usage in data['usages'][:5]:
        print(f"   • {usage['feature_name']} (line {usage['line_number']})")
        print(f"     {usage['code_snippet']}")

except Exception as e:
    print(f"❌ Error: {e}")

🔍 Demo 2: Scanning Code for Features

Code to scan:
------------------------------------------------------------

.header {
    backdrop-filter: blur(10px);
    background: rgba(255, 255, 255, 0.8);
}

.container:has(> .active) {
    background: blue;
}

@container (min-width: 400px) {
    .card {
        display: grid;
        grid-template-columns: subgrid;
    }
}

.scroll-container {
    scroll-snap-type: y mandatory;
}

------------------------------------------------------------
INFO:     34.106.104.187:0 - "POST /scan HTTP/1.1" 200 OK

✅ Scan Results:
   Features Found: 5
   Unique Features: 5
   Global Support: 72.4%
   Affected Users: 27.6%

📊 Features by Risk:
   🔴 High Risk: None
   🟡 Medium Risk: subgrid
   🟢 Low Risk: :has(), backdrop-filter, container-queries, scroll-snap

📋 Detected Features:
   • backdrop-filter (line 3)
     backdrop-filter: blur(10px);
   • scroll-snap (line 19)
     scroll-snap-type: y mandatory;
   • :has() (line 7)
     .container:has(> .active) {


In [ ]:
print("⚠️ Demo 3: Feature Risk Assessment\n")

features_to_check = [":has()", "subgrid", "backdrop-filter"]

for feature_name in features_to_check:
    try:
        response = requests.get(f"{public_url}/risk/{feature_name}")
        risk = response.json()

        risk_emoji = "🟢" if risk['risk_level'] < 3 else "🟡" if risk['risk_level'] < 6 else "🔴"

        print(f"{risk_emoji} {risk['feature_name']}")
        print(f"   Support: {risk['global_support']}%")
        print(f"   Risk Score: {risk['risk_level']:.1f}/10")
        print(f"   Safe Year: {risk['safe_year']}")
        print(f"   Browsers: {risk['browsers']}")
        print(f"   💡 {risk['recommendation']}")
        if risk['alternatives']:
            print(f"   🔄 Alternatives: {', '.join(risk['alternatives'])}")
        print(f"   📍 Top Affected Markets:")
        for market, pct in risk['affected_markets'][:3]:
            print(f"      • {market}: {pct:.1f}% affected")
        print()

    except Exception as e:
        print(f"❌ Error checking {feature_name}: {e}\n")


⚠️ Demo 3: Feature Risk Assessment

INFO:     34.106.104.187:0 - "GET /risk/%3Ahas%28%29 HTTP/1.1" 200 OK
🟢 :has()
   Support: 87.3%
   Risk Score: 1.3/10
   Safe Year: 2024
   Browsers: {'chrome': '105', 'safari': '15.4', 'firefox': '121'}
   💡 ✅ Safe to use. :has() has excellent support (87.3%)
   🔄 Alternatives: :not() combinations, JS-based selectors
   📍 Top Affected Markets:
      • India: 11.4% affected
      • China: 10.8% affected
      • Brazil: 10.2% affected

INFO:     34.106.104.187:0 - "GET /risk/subgrid HTTP/1.1" 200 OK
🟡 subgrid
   Support: 72.4%
   Risk Score: 4.1/10
   Safe Year: 2026
   Browsers: {'chrome': '117', 'safari': '16', 'firefox': '71'}
   💡 ⚠️ Use with caution. Consider progressive enhancement or polyfills
   🔄 Alternatives: nested grids, flexbox
   📍 Top Affected Markets:
      • India: 24.8% affected
      • China: 23.5% affected
      • Brazil: 22.1% affected

INFO:     34.106.104.187:0 - "GET /risk/backdrop-filter HTTP/1.1" 200 OK
🟢 backdrop-filter
   

In [ ]:
print("🎨 Generating React Dashboard HTML...\n")

# Create standalone HTML dashboard
dashboard_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WFIP Dashboard - Colab</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.js"></script>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-gray-900 text-white">
    <div class="container mx-auto px-6 py-8">
        <div class="bg-gradient-to-r from-purple-600 to-blue-600 rounded-2xl p-8 mb-8">
            <h1 class="text-4xl font-bold mb-2">WFIP Dashboard</h1>
            <p class="text-white/80">Web Feature Intelligence Platform - Connected to Colab Backend</p>
            <p class="text-sm text-white/60 mt-2">API: <span id="api-url"></span></p>
        </div>

        <div class="grid grid-cols-1 md:grid-cols-3 gap-6 mb-8">
            <div class="bg-gray-800 rounded-xl p-6">
                <h3 class="text-gray-400 text-sm mb-2">Features Tracked</h3>
                <p class="text-4xl font-bold" id="feature-count">-</p>
            </div>
            <div class="bg-gray-800 rounded-xl p-6">
                <h3 class="text-gray-400 text-sm mb-2">Avg Support</h3>
                <p class="text-4xl font-bold text-green-400" id="avg-support">-</p>
            </div>
            <div class="bg-gray-800 rounded-xl p-6">
                <h3 class="text-gray-400 text-sm mb-2">High Risk</h3>
                <p class="text-4xl font-bold text-red-400" id="high-risk-count">-</p>
            </div>
        </div>

        <div class="bg-gray-800 rounded-xl p-6 mb-8">
            <h2 class="text-2xl font-bold mb-4">Feature Support Chart</h2>
            <canvas id="supportChart" height="100"></canvas>
        </div>

        <div class="bg-gray-800 rounded-xl p-6">
            <h2 class="text-2xl font-bold mb-4">All Features</h2>
            <div id="features-list" class="space-y-3"></div>
        </div>
    </div>

    <script>
        const API_URL = 'API_URL_PLACEHOLDER';
        document.getElementById('api-url').textContent = API_URL;

        async function loadData() {
            try {
                const response = await fetch(`${API_URL}/features`);
                const data = await response.json();

                document.getElementById('feature-count').textContent = data.total;

                const supports = data.features.map(f => f.global_support);
                const avgSupport = (supports.reduce((a, b) => a + b, 0) / supports.length).toFixed(1);
                document.getElementById('avg-support').textContent = avgSupport + '%';

                const highRisk = data.features.filter(f => f.global_support < 80).length;
                document.getElementById('high-risk-count').textContent = highRisk;

                // Chart
                const ctx = document.getElementById('supportChart').getContext('2d');
                new Chart(ctx, {
                    type: 'bar',
                    data: {
                        labels: data.features.map(f => f.name),
                        datasets: [{
                            label: 'Global Support %',
                            data: data.features.map(f => f.global_support),
                            backgroundColor: data.features.map(f =>
                                f.global_support >= 95 ? '#10b981' :
                                f.global_support >= 80 ? '#f59e0b' : '#ef4444'
                            )
                        }]
                    },
                    options: {
                        responsive: true,
                        plugins: { legend: { labels: { color: '#fff' } } },
                        scales: {
                            y: { ticks: { color: '#9ca3af' }, grid: { color: '#374151' } },
                            x: { ticks: { color: '#9ca3af' }, grid: { display: false } }
                        }
                    }
                });

                // Features list
                const listEl = document.getElementById('features-list');
                data.features.forEach(f => {
                    const color = f.global_support >= 95 ? 'green' : f.global_support >= 80 ? 'yellow' : 'red';
                    listEl.innerHTML += `
                        <div class="border border-gray-700 rounded-lg p-4">
                            <div class="flex justify-between items-start">
                                <div>
                                    <code class="text-blue-400 text-lg">${f.name}</code>
                                    <p class="text-sm text-gray-400 mt-1">${f.baseline_status.replace('_', ' ')}</p>
                                </div>
                                <span class="text-${color}-400 text-2xl font-bold">${f.global_support}%</span>
                            </div>
                            <div class="mt-3 text-sm text-gray-400">
                                <p>Browsers: ${Object.entries(f.browsers).map(([b, v]) => `${b} ${v}`).join(', ')}</p>
                            </div>
                        </div>
                    `;
                });

            } catch (error) {
                console.error('Error loading data:', error);
                document.body.innerHTML += '<div class="fixed top-4 right-4 bg-red-500 text-white px-6 py-4 rounded-lg">Error connecting to API</div>';
            }
        }

        loadData();
    </script>
</body>
</html>
"""

# Save dashboard
dashboard_html = dashboard_html.replace('API_URL_PLACEHOLDER', public_url)

with open('/content/wfip_dashboard.html', 'w') as f:
    f.write(dashboard_html)

print("✅ Dashboard HTML generated!")
print(f"📁 Saved to: /content/wfip_dashboard.html")
print("\n🎯 To view the dashboard:")
print("1. Download the HTML file from Colab files panel (left sidebar)")
print("2. Open it in your browser")
print("3. Or use the command below to display it inline\n")

# Display dashboard inline in Colab
from IPython.display import HTML, display
display(HTML(f'<iframe src="{public_url}/docs" width="100%" height="600px"></iframe>'))

print("📚 Interactive API documentation shown above!")
print(f"🔗 Dashboard URL: {public_url}")

🎨 Generating React Dashboard HTML...

✅ Dashboard HTML generated!
📁 Saved to: /content/wfip_dashboard.html

🎯 To view the dashboard:
1. Download the HTML file from Colab files panel (left sidebar)
2. Open it in your browser
3. Or use the command below to display it inline



/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


📚 Interactive API documentation shown above!
🔗 Dashboard URL: https://raymond-overheady-allopathically.ngrok-free.dev


In [ ]:
print("\n⏰ Server is running...")
print("💡 The server will stay active while this cell is running")
print("🛑 Press the Stop button in Colab to shutdown")
print("\n" + "=" * 60)

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")
    ngrok.disconnect(public_url)


⏰ Server is running...
💡 The server will stay active while this cell is running
🛑 Press the Stop button in Colab to shutdown


🛑 Server stopped!


PyngrokNgrokURLError: ngrok client exception, URLError: [Errno 111] Connection refused